# Data Preprocessing for Spotify Emotion Classification

Machine learning engineers are looking to classify songs into one of four categories:
Sad (0), happy (1), energetic (2), and calm (3). For each song, the following features have been collected.

Acousticness: A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.

Danceability: Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.

Energy: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.

Instrumentalness: Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.

Liveness: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides a strong likelihood that the track is live.

Loudness: the overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing the relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db.

Speechiness: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audiobook, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.

Valence: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

Tempo: The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, the tempo is the speed or pace of a given piece and derives directly from the average beat duration.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
songs = pd.read_csv('/Users/carterdemars/Desktop/archive/spotify_mood_full.csv', index_col=0)

In [3]:
X = songs.drop('labels', axis=1)
y = songs['labels']

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=101)

In [5]:
y_train.value_counts()

labels
1    74500
0    57441
2    32945
3    29670
Name: count, dtype: int64

In [6]:
y_test.value_counts()

labels
1    31929
0    24617
2    14120
3    12716
Name: count, dtype: int64

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, RidgeClassifier

In [8]:
knn = LGBMClassifier()

In [9]:
knn.fit(X_train, y_train)

LGBMClassifier()

In [10]:
knn.score(X_test, y_test)

0.960710944808232

In [11]:
mask = X_train.sample(5000).index

In [12]:
knn.fit(X_train.loc[mask], y_train.loc[mask])

LGBMClassifier()

In [13]:
knn.score(X_test, y_test)

0.9243601736585834

In [14]:
sample_data = X_train.loc[mask]
sample_data['label'] = y_train.loc[mask]

In [15]:
duplicate_data_index = sample_data.sample(3000).index

In [16]:
duplicate_data = pd.concat([sample_data, sample_data.loc[duplicate_data_index]], ignore_index=True)

In [17]:
duplicate_data.head()

,duration (ms),danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,spec_rate,label
0,264000.0,0.411,0.9910,-0.905,0.1760,0.00587,0.792000,0.437,0.3530,112.239,6.666667e-07,2
1,234147.0,0.468,0.4720,-7.318,0.0270,0.80900,0.000004,0.106,0.2470,133.056,1.153122e-07,0
2,265667.0,0.254,0.1940,-17.177,0.0349,0.93700,0.881000,0.116,0.0391,115.639,1.313675e-07,3
3,142613.0,0.377,0.0542,-24.809,0.0378,0.99400,0.917000,0.127,0.1460,141.039,2.650530e-07,3
4,189427.0,0.587,0.3130,-18.267,0.0431,0.85400,0.826000,0.298,0.6020,120.021,2.275283e-07,3


In [18]:
duplicate_data.shape

(8000, 12)

In [19]:
duplicate_data.drop_duplicates().shape

(4999, 12)

In [44]:
duplicate_data['random_noise'] = duplicate_data['energy'] + duplicate_data['instrumentalness'] + np.random.random(duplicate_data.shape[0])
duplicate_data['random_noise2'] = duplicate_data['valence'] + duplicate_data['tempo']/100 + np.random.random(duplicate_data.shape[0])

In [45]:
duplicate_data.head()

,duration (ms),danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,spec_rate,label,random_noise,random_noise2
0,264000.0,0.411,0.9910,-0.905,0.1760,0.00587,0.792000,0.437,0.3530,112.239,6.666667e-07,2,2.407958,2.211753
1,234147.0,0.468,0.4720,-7.318,0.0270,0.80900,0.000004,0.106,0.2470,133.056,1.153122e-07,0,1.070228,1.907492
2,265667.0,0.254,0.1940,-17.177,0.0349,0.93700,0.881000,0.116,0.0391,115.639,1.313675e-07,3,1.548747,1.473796
3,142613.0,0.377,0.0542,-24.809,0.0378,0.99400,0.917000,0.127,0.1460,141.039,2.650530e-07,3,1.759897,2.384712
4,189427.0,0.587,0.3130,-18.267,0.0431,0.85400,0.826000,0.298,0.6020,120.021,2.275283e-07,3,1.351590,2.035159


In [54]:
test = DecisionTreeClassifier(min_samples_split=1)
dup_train, dup_test, dup_y_train, dup_y_test = train_test_split(duplicate_data.drop('label', axis=1), duplicate_data['label'], test_size=0.3, random_state=101)
test.fit(dup_train, dup_y_train)

DecisionTreeClassifier(min_samples_split=1)

In [55]:
test.score(dup_test, dup_y_test)

0.9158333333333334

In [48]:
test = RidgeClassifier()

In [49]:
test.fit(dup_train, dup_y_train)

RidgeClassifier()

In [50]:
test.score(dup_test, dup_y_test)

0.7729166666666667

In [56]:
test2 = DecisionTreeClassifier(min_samples_split=1)
test2.fit(X_train, y_train)
test2.score(X_test, y_test)

0.9033844234966779

The script will not run successfully until you
1) drop or interpolate missing values
2) convert label from categorical to numeric

To improve the performance of the model, we recommend that you
1) Check for duplicate values
2) Check for outliers and invalid values
3) Check for unnecessary features